In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

## Main Function

In [2]:
def open_excel(fname, sheet):
    df = pd.read_excel('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/excel_data/' + fname + '.xlsx', sheet_name=sheet)
    return df

def save_csv(df, fname):
    df.to_csv('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/csv_data/UNICEF/' + fname, index=False)

def open_csv(fname):
    df = pd.read_csv('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/csv_data/UNICEF/' + fname)
    return df

## HPV Vaccine Coverage

In [ ]:
hpvdf = open_excel('HPV_estimates-2019revision', 'DATA_HPV_SERIES')
hpvdf = hpvdf.drop(hpvdf.iloc[:, -3:].columns, axis=1)

In [ ]:
hpvdf = hpvdf[(df['year'] == 2019) & (df['sex'] != 'Male')]
hpvdf

In [ ]:
df = df[df['value_str'] != '-']
hpvdf

In [ ]:
save_csv(hpvdf, 'world_hpv_coverage.csv')

In [ ]:
# Take only HPV of female with complete dose
hpvdf_fc = open_csv('world_hpv_coverage.csv')

In [ ]:
hpvdf_fc = hpvdf_fc[hpvdf_fc['indicator'] == 'prHPVc_F']

In [ ]:
hpvdf_fc[['value_str']] = hpvdf_fc[['value_str']].apply(lambda x: x.str.strip('%' and '%†' and '%*'))

In [ ]:
hpvdf_fc['value_str'] = hpvdf_fc['value_str'].astype('float64')

In [ ]:
save_csv(hpvdf_fc, 'world_hpv_coverage_prHPVc_F.csv')

## Vaccine Coverage for DTP3 and POL3

In [3]:
dtp3 = open_excel('immunization_coverage_2019', sheet='DTP3')
pol3 = open_excel('immunization_coverage_2019', sheet='POL3')

In [4]:
dtp3 = dtp3.iloc[:, :24]
dtp3 = dtp3.dropna()
dtp3 = dtp3.drop('iso3', axis=1)
dtp3 = pd.melt(dtp3, id_vars=['unicef_region','country', 'vaccine'], var_name='Year', value_name='Coverage %')

In [5]:
pol3 = pol3.iloc[:, :24]
pol3 = pol3.dropna()
pol3 = pol3.drop('iso3', axis=1)
pol3 = pd.melt(pol3, id_vars=['unicef_region','country', 'vaccine'], var_name='Year', value_name='Coverage %')

In [ ]:
save_csv(dtp3, 'dtp3.csv')
save_csv(pol3, 'pol3.csv')

### Plot Coverage % for different region and disease

In [6]:
dtp3 = open_excel('immunization_coverage_2019', sheet='DTP3')
pol3 = open_excel('immunization_coverage_2019', sheet='POL3')
hepb3 = open_excel('immunization_coverage_2019', sheet='HEPB3')
hib3 = open_excel('immunization_coverage_2019', sheet='HIB3')
mcv1 = open_excel('immunization_coverage_2019', sheet='MCV1')

In [7]:
def check_df(df, col):
    df.iloc[:, :col].dropna().info()

def preprocess_df(df):
    df = df.iloc[:, :15].drop('iso3', axis=1).dropna()
    df = pd.melt(df, id_vars=['unicef_region','country', 'vaccine'], var_name='Year', value_name='Coverage %')
    df.rename(columns={'unicef_region':'Region', 'country':'Country', 'vaccine':'Vaccine Type'}, inplace=True)
    return df

In [63]:
dtp3_10y = preprocess_df(dtp3)
pol3_10y = preprocess_df(pol3)
hepb3_10y = preprocess_df(hepb3)
hib3_10y = preprocess_df(hib3)
mcv1_10y = preprocess_df(mcv1)

In [217]:
fig = plt.figure(figsize=(20,10))
gs = GridSpec(2,3, wspace=0.2, hspace=0.5, figure=fig)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[1, 0])
ax5 = fig.add_subplot(gs[1, 1])
sns.set_style("white")
sns.lineplot(x='Year', y='Coverage %', hue='Region', data=dtp3_10y, ci=None, ax=ax1)
ax1.set_title('DTP3 Coverage % 2009-2019')
ax1.legend().set_visible(False)
sns.lineplot(x='Year', y='Coverage %', hue='Region', data=pol3_10y, ci=None, ax=ax2)
ax2.set_title('POL3 Coverage % 2009-2019')
ax2.legend().set_visible(False)
sns.lineplot(x='Year', y='Coverage %', hue='Region', data=hepb3_10y, ci=None, ax=ax3)
ax3.set_title('HEPB3 Coverage % 2009-2019')
ax3.legend().set_visible(False)
sns.lineplot(x='Year', y='Coverage %', hue='Region', data=hib3_10y, ci=None, ax=ax4)
ax4.set_title('HIB3 Coverage % 2009-2019')
ax4.legend().set_visible(False)
sns.lineplot(x='Year', y='Coverage %', hue='Region', data=mcv1_10y, ci=None, ax=ax5)
ax5.set_title('MCV1 Coverage % 2009-2019')
plt.legend(bbox_to_anchor=(1.3, 1))
fig.suptitle('Global Coverage % ''by Regions from 2009 to 2019 for Different Types of Vaccines')
fig.savefig('/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/global_vaccine_coverage_10y.png')
plt.close()

## Region HDI

In [47]:
hdi_df = open_excel('hdro_statistical_data_table_1', sheet='HDI')

In [48]:
hdi_df = hdi_df.rename({'HDI':'Human Development Index (HDI)'}, axis=1)

In [61]:
fig, ax = plt.subplots(figsize=(18,14))
sns.scatterplot(x="Human Development Index (HDI)", y='Region', hue='Region', data=hdi_df, ax=ax)
ax.legend().set_visible(False)
fig.suptitle("Human Development Index of Every Country by Region in 2018", fontsize=16)
fig.savefig("/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/hdi.png")
plt.close()

## Gavi Support

In [87]:
not_eligible = hib3_10y[(hib3_10y['Country'] == 'South Africa')]
eligible = hib3_10y[(hib3_10y['Country'] == 'Kenya')]

In [88]:
gavi_df = pd.concat([not_eligible, eligible])

In [142]:
fig, ax = plt.subplots(figsize=(10,10))
sns.lineplot(x='Year', y='Coverage %', hue='Country', data=gavi_df, ax=ax, markers=True, dashes=False, palette=['#536B8E', '#F79266'], hue_order=['Kenya', 'South Africa'], style='Country')
fig.suptitle('Difference in HIB3 Coverage (%) between Eligible and Non-eligible Country for Gavi Support')
fig.savefig("/Users/Ming/Documents/GitHub/wellcome_dissertation/WellcomePy/py_image/gavi_support.png")
plt.close()